In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from helpers import *

y, tX, ids = load_csv_data('data/train.csv')
_, tX_test, ids_test = load_csv_data('data/test.csv')

Manage missing values

In [3]:
tX = missing_values(tX)
tX_test = missing_values(tX_test)

In [5]:
from methods import *

loss, weights = least_squares(y,tX)

In [6]:
weights

array([-9.41956998e-01,  2.67387363e-04, -7.36995344e-03, -6.64827271e-03,
        6.10889176e-05,  3.61359462e-01, -1.81816979e-03, -3.12258510e+00,
       -2.29207107e-01,  1.09625834e-01,  3.13117259e+00, -7.33715328e-04,
       -4.70005529e-04,  3.13594939e+00, -4.95627367e-04,  1.28485571e-03,
        3.20227772e-03,  4.56072743e-04, -5.55826323e-04,  8.54993742e-02,
        3.12201329e+00])

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [7]:
OUTPUT_PATH = 'data/firstsubmission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)